In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
df= pd.read_excel('cleandatazscore2911.xlsx', index_col= 0)

In [3]:
# Define the function
def calculate_average(df, columns, new_column_name):
    """
    Calculates the mean of the specified columns and adds a new column with the result.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns (list): The list of column names to calculate the mean.
    new_column_name (str): The name of the new column to store the averages.
    
    Returns:
    pd.DataFrame: The DataFrame with the new column added.
    """
    df[new_column_name] = df[columns].mean(axis=1)
    return df

In [4]:
reading = ['ASRREA01', 'ASRREA02', 'ASRREA03', 'ASRREA04', 'ASRREA05'] 
literary_purpose = ['ASRLIT01', 'ASRLIT02', 'ASRLIT03', 'ASRLIT04', 'ASRLIT05']
informational_purpose=['ASRINF01', 'ASRINF02', 'ASRINF03', 'ASRINF04', 'ASRINF05']
interpreting_process= ['ASRIIE01', 'ASRIIE02', 'ASRIIE03', 'ASRIIE04', 'ASRIIE05']
straightforward_process = ['ASRRSI01', 'ASRRSI02', 'ASRRSI03', 'ASRRSI04', 'ASRRSI05']

In [5]:
df = calculate_average(df, reading, 'reading_avg')
df = calculate_average(df, literary_purpose, 'literary_purpose_avg')
df = calculate_average(df, informational_purpose, 'informational_purpose_avg')
df = calculate_average(df, interpreting_process, 'interpreting_process_avg')
df = calculate_average(df, straightforward_process, 'straightforward_process_avg')

In [6]:
averages = ['reading_avg', 'literary_purpose_avg', 'informational_purpose_avg','interpreting_process_avg', 'straightforward_process_avg']

I want to choose a dataset from the filtered one ( z score etc. ) and then do these correlation thingies for the numeric data. 

In [7]:
df['avgscore_binned'] = pd.cut(df['avgscore'], bins=10)

In [8]:
from scipy.stats import chi2_contingency

In [9]:
# Create a contingency table
contingency_table = pd.crosstab(df['ASBH02A'], df['avgscore_binned'])

In [10]:
contingency_table.head()

avgscore_binned,"(130.542, 137.546]","(137.546, 144.481]","(144.481, 151.416]","(151.416, 158.351]","(158.351, 165.286]","(165.286, 172.221]","(172.221, 179.155]","(179.155, 186.09]","(186.09, 193.025]","(193.025, 199.96]"
ASBH02A,,,,,,,,,,
No,0,2,1,3,4,1,5,5,4,6
Yes,21,31,32,49,63,56,60,87,78,109


In [11]:
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

# Create a confusion matrix
confusion_matrix = pd.crosstab(df['ASBH02A'], df['avgscore_binned'])

# Calculate Cramér's V
cramers_v_value = cramers_v(confusion_matrix)
print(f"Cramér's V: {cramers_v_value}")

Cramér's V: 0.07989243318299707



Cramér's V is a statistical measure used to assess the strength of association between two nominal (categorical) variables. It is derived from the Chi-square statistic and ranges from 0 to 1, where:

0 indicates no association between the variables.
1 indicates a perfect association between the variables.
The value you provided, 0.077, is relatively close to 0, indicating a very weak association between the two variables.

Here's a general interpretation of Cramér's V:

0 to 0.1: Little or no association\
0.1 to 0.3: Weak association\
0.3 to 0.5: Moderate association\
Above 0.5: Strong association\

Since your Cramér's V is around 0.077, it suggests that the relationship between the two categorical variables in your analysis is negligible or very weak.

In [12]:
# Perform Chi-Square Test of Independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

In [13]:
print(f"Chi-Square Test Statistic: {chi2}")
print(f"P-value: {p}")

Chi-Square Test Statistic: 3.9381881428980847
P-value: 0.9154334828579949


1. Chi-Square Test Statistic: 245.83458534739597
The Chi-Square test statistic quantifies how much the observed frequencies in your data deviate from the expected frequencies under the assumption that the two categorical variables are independent.
A higher Chi-Square value generally indicates a greater difference between the observed and expected frequencies, suggesting a stronger association between the variables.
In your case, the Chi-Square statistic is quite large, indicating a noticeable deviation from the expected distribution if the variables were independent.
2. P-value: 7.553360938216542e-48
The P-value represents the probability of observing a Chi-Square statistic as extreme as, or more extreme than, the one calculated, under the null hypothesis (which states that the two variables are independent).
A very small P-value (like the one here, which is effectively 0.000...0000755) indicates that the observed association is highly unlikely to have occurred by chance.
Common thresholds for significance are 0.05, 0.01, and 0.001. Your P-value is far smaller than any of these, meaning you can reject the null hypothesis with extremely high confidence, suggesting a statistically significant association between the two variables.
Summary
Chi-Square Test Statistic (245.83): Indicates a substantial deviation from what would be expected under independence.
P-value (~0): Shows that this deviation is extremely unlikely to be due to random chance, indicating a statistically significant association.
However, the earlier Cramér's V value of 0.077 suggests that while the association is statistically significant, it is not strong. This can happen when the sample size is large, leading to statistically significant results even for weak associations.